In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
from os.path import isfile

In [76]:
def get_url_resp(country="Worldwide"):
    base_url = "https://www.worldometers.info/coronavirus/"
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0"}
    if country =="Worldwide":
        url = base_url
    else:
        url =base_url+"country/"+country+"/"
    response = requests.get(url, headers = headers)
    html = response.text
    return BeautifulSoup(html,"html.parser")

def format_case_numbers(soup,country="Worldwide"):
    if country == "Worldwide":
        soup_selector = soup.select(".col-md-6")
        val = [soup_selector[2].find_all(type="text/javascript")[0],soup_selector[3].find_all(type="text/javascript")[0]]
    else:
        soup_selector = soup.find_all("script")
        new_soup =[]
        for i in range(len(soup_selector)):
            if 'categories' in str(soup_selector[i]):
                new_soup.append(soup_selector[i])
        val = new_soup#[:2]
    return val

def get_right_element(val,Status = "Cases"):
    string1 = f"'Total Coronavirus {Status}'"
    new_soup =[]
    for i in range(len(val)):
        if string1 in str(val[i]):
            new_soup.append(val[i])
    return new_soup

def get_both_elements(val):#,country="Worldwide"):
    # if country=="Worldwide":
    #     return val
    # else:
    statuses_tuple = ("Cases","Deaths")
    new_soup = []
    for sta in statuses_tuple:
        new_soup.append(get_right_element(val,sta))
    return new_soup

def get_case_numbers_from_soup(val):#,country="Worldwide",status="Cases"):
    #dates = val.get_text().rsplit("categories",1)[1].split("[")[1].split("]")[0]
    dates = val.get_text().rsplit("categories")[1].split("[")[1].split("]")[0]
    case_num = val.get_text().rsplit("data")[1].split("[")[1].split("]")[0]
    case_num_formatted=case_num.split(',')
    dates_formatted=dates.split(',')
    case_num_lst=[]
    date_lst = []
    # for lst in case_num_lst,date_lst:
    #     lst.append(country)
    #     lst.append(status)

    for i in range(len(case_num_formatted)):
        if case_num_formatted[i] == "null":
            case_num_input = 0
        else:
            case_num_input = int(case_num_formatted[i])
        case_num_lst.append(case_num_input)

    for i in dates_formatted:
        date_lst.append(i.replace('"',''))
    return zip(date_lst,case_num_lst)

def get_case_dictionary(country="Worldwide"):#,status="Cases"):
    soup = get_url_resp(country)
    new_soup1 = format_case_numbers(soup,country)
    val = get_both_elements(new_soup1)
    get_right_element(val,Status = "Cases")
    statuses_tuple = ("Cases","Deaths")
    statuses = {
        "Cases":0,
        "Deaths":1,
    }
    lst=[]
    for sta in statuses_tuple:
        lst.append(dict(get_case_numbers_from_soup(val[statuses[sta]])))
    return lst
    #return dict(get_case_numbers_from_soup(val[statuses[status]]))#,country,status)
    # return get_case_numbers_from_soup(val[statuses[status]],country,status)


In [77]:
Countries = ("Worldwide","US","Italy","China","Spain","Germany","France","UK","India","South-Korea")
statuses_tuple = ("Cases","Deaths")
lst=[[],[]]
for country in Countries:
    for sta in statuses_tuple:
        lst[0].append(country)
        lst[1].append(sta)
### WORKS TO HERE ###
# lst = [["Worldwide","US"],["Cases","Cases"]]

data=[]
for country in Countries:
    print(country)
    country_data=get_case_dictionary(country=country)
    print(country_data)
    data.extend(country_data)
print(data)

Worldwide


TypeError: 'NoneType' object is not callable

In [ ]:
tuples = list(zip(*lst))
index = pd.MultiIndex.from_tuples(tuples)
cols = list(data[0].keys())
data_types = {val:pd.Int64Dtype() for val in cols}

In [69]:
country="Worldwide"
soup = get_url_resp(country)
new_soup1 = format_case_numbers(soup,country)
val = get_both_elements(new_soup1)

# print(list(get_case_numbers_from_soup(val[0])))
# print(list(get_case_numbers_from_soup(val[1])))
#"'Total Coronavirus Cases'"
#"'Total Coronavirus Deaths'"
# print(val)
print(val.get_text())
# print(val[3])
# val=val[0]
# # print(val)
# dates = val.get_text().rsplit("categories")[1].split("[")[1].split("]")[0]
# # print(dates)
# case_num = val.get_text().rsplit("data",1)[1].split("[")[1].split("]")[0]
# case_num_formatted=case_num.split(',')
# dates_formatted=dates.split(',')
# print(dates_formatted)
# case_num_lst=[]
# date_lst = []

AttributeError: 'list' object has no attribute 'get_text'